In [11]:
%load_ext autoreload
%autoreload 2

import sys
import os
import importlib
import ipywidgets as widgets
from IPython.display import clear_output
import matplotlib.pyplot as plt
import numpy as np

if os.getcwd() not in sys.path:
    sys.path.append(os.getcwd())

import src.config
import src.transform
import src.data_loader
import src.metrics
import src.codec


# Примусово оновлюємо код з диску в пам'ять
importlib.reload(src.config)
importlib.reload(src.transform)
importlib.reload(src.data_loader)
importlib.reload(src.metrics)
importlib.reload(src.codec)


from src.codec import BPGCodec
from src.config import VSTConfig
from src.transform import VarianceStabilizer
from src.data_loader import SyntheticGenerator, ImageLoader
from src.metrics import NoiseEstimator


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
class VSTExplorerApp:
    def __init__(self, default_noised='data/NOISED.tiff', default_original='data/ORIGINAL.tiff'):
        # --- State ---
        self.default_noised = default_noised
        self.default_original = default_original
        
        self.cached_gen_data = None
        self.last_gen_noise_val = -1
        
        # --- UI Initialization ---
        self._init_widgets()
        self._init_layout()
        
    def _init_widgets(self):
        style = {'description_width': 'initial'}
        
        self.w_source = widgets.Dropdown(
            options=[('Генератор (Simulation)', 'gen'), ('Файл (File)', 'file')],
            value='gen', description='Джерело:', style=style
        )
        
        self.w_path = widgets.Text(
            value=self.default_noised, placeholder='path/to/image.tiff',
            description='Шлях:', style=style, layout=widgets.Layout(width='300px')
        )
        
        self.w_a = widgets.FloatSlider(value=8.39, min=1.0, max=20.0, step=0.1, 
                                     description='Param a:', style=style, continuous_update=False)
        self.w_b = widgets.FloatSlider(value=1.2, min=1.05, max=5.0, step=0.05, 
                                     description='Param b:', style=style, continuous_update=False)
        
        self.w_noise_gen = widgets.FloatSlider(value=0.25, min=0.01, max=1.0, step=0.01, 
                                             description='Gen Noise:', style=style, continuous_update=False)

        # Оновлена кнопка: тепер вона називається "Оновити / Скинути"
        self.btn_reset = widgets.Button(description='Force Refresh', icon='refresh', button_style='warning')
        self.btn_reset.on_click(self.reset_params)
        
        self.out_plot = widgets.Output()

    def _init_layout(self):
        row_ctrl = widgets.HBox([self.w_source, self.w_noise_gen, self.w_path])
        row_params = widgets.HBox([self.w_a, self.w_b, self.btn_reset])
        
        def on_mode_change(change):
            mode = change['new']
            if mode == 'gen':
                self.w_path.layout.display = 'none'
                self.w_noise_gen.layout.display = 'flex'
            else:
                self.w_path.layout.display = 'flex'
                self.w_noise_gen.layout.display = 'none'
        
        self.w_source.observe(on_mode_change, names='value')
        on_mode_change({'new': self.w_source.value})

        for w in [self.w_source, self.w_path, self.w_a, self.w_b, self.w_noise_gen]:
            w.observe(self.update, names='value')
            
        display(widgets.VBox([row_ctrl, row_params, self.out_plot]))
        self.update()

    def reset_params(self, b):
        # 1. Скидаємо параметри VST на дефолтні
        self.w_a.value = 8.39
        self.w_b.value = 1.2
        
        # 2. !!! ВАЖЛИВО: Очищаємо кеш генератора !!!
        self.cached_gen_data = None
        self.last_gen_noise_val = -1
        
        # 3. Викликаємо оновлення вручну
        self.update()

    def get_data_pair(self):
        if self.w_source.value == 'gen':
            current_noise = self.w_noise_gen.value
            # Тепер ми генеруємо дані, якщо кеш пустий (після Reset) АБО змінився слайдер
            if self.cached_gen_data is None or current_noise != self.last_gen_noise_val:
                # Тут Python викличе оновлений код з файлу (завдяки %autoreload)
                self.cached_gen_data = SyntheticGenerator.get_data(noise_level=current_noise)
                self.last_gen_noise_val = current_noise
            return self.cached_gen_data
            
        else: # File mode
            try:
                path_in = self.w_path.value
                img_noised = ImageLoader.load_file(path_in)
                img_clean = None
                
                if os.path.exists(self.default_original):
                     img_clean = ImageLoader.load_file(self.default_original)
                
                if img_clean is None:
                    dir_name = os.path.dirname(path_in)
                    candidate_path = os.path.join(dir_name, 'ORIGINAL.tiff')
                    if os.path.exists(candidate_path):
                        img_clean = ImageLoader.load_file(candidate_path)

                return img_clean, img_noised
            except Exception:
                return None, None

    def update(self, change=None):
        img_clean, img_noised = self.get_data_pair()
        
        with self.out_plot:
            clear_output(wait=True)
            if img_noised is None:
                print(f"File error: {self.w_path.value}")
                return

            config = VSTConfig(a=self.w_a.value, b=self.w_b.value)
            vst = VarianceStabilizer(config)
            
            img_log_noised = vst.forward(img_noised)
            img_restored = vst.inverse(img_log_noised)
            
            sigma_blind = NoiseEstimator.estimate_blind_sigma(img_log_noised)
            sigma_exact = 0.0
            noise_map_vector = None
            
            if img_clean is not None:
                if img_clean.shape == img_noised.shape:
                    img_log_clean = vst.forward(img_clean)
                    sigma_exact = NoiseEstimator.calculate_exact_sigma(img_log_noised, img_log_clean)
                    noise_map_vector = (img_log_noised - img_log_clean).flatten()

            fig = plt.figure(figsize=(14, 8), constrained_layout=True)
            gs = fig.add_gridspec(2, 2)
            
            ax_in = fig.add_subplot(gs[0, 0])
            ax_log = fig.add_subplot(gs[0, 1])
            ax_out = fig.add_subplot(gs[1, 0])
            ax_hist = fig.add_subplot(gs[1, 1])
            
            vmin, vmax = np.percentile(img_noised, 1), np.percentile(img_noised, 99)
            
            im0 = ax_in.imshow(img_noised, cmap='gray', vmin=vmin, vmax=vmax)
            ax_in.set_title(f"Input\nMin: {img_noised.min():.2f}, Max: {img_noised.max():.2f}")
            plt.colorbar(im0, ax=ax_in, fraction=0.046)
            
            im1 = ax_log.imshow(img_log_noised, cmap='viridis')
            ax_log.set_title(f"Log Domain\nBlind Sigma: {sigma_blind:.4f}")
            plt.colorbar(im1, ax=ax_log, fraction=0.046)
            
            im2 = ax_out.imshow(img_restored, cmap='gray', vmin=vmin, vmax=vmax)
            mse = np.mean((img_noised - img_restored)**2)
            ax_out.set_title(f"Restored\nMSE: {mse:.2e}")
            plt.colorbar(im2, ax=ax_out, fraction=0.046)
            
            if noise_map_vector is not None:
                ax_hist.hist(noise_map_vector, bins=100, density=True, alpha=0.6, color='dodgerblue', label='Actual')
                x_axis = np.linspace(noise_map_vector.min(), noise_map_vector.max(), 100)
                mean_val = np.mean(noise_map_vector)
                pdf = (1 / (sigma_exact * np.sqrt(2 * np.pi))) * np.exp(-0.5 * ((x_axis - mean_val) / sigma_exact)**2)
                ax_hist.plot(x_axis, pdf, 'r--', linewidth=2, label=f'Gauss $\sigma={sigma_exact:.4f}$')
                ax_hist.set_title("Noise Histogram")
                ax_hist.legend()
            else:
                ax_hist.text(0.5, 0.5, "No Reference", ha='center')
                ax_hist.axis('off')
            
            plt.show()

app = VSTExplorerApp()

In [13]:
# --- IMPORTS FOR BPG APP ---
import ipywidgets as widgets
from IPython.display import display, clear_output
import matplotlib.pyplot as plt
import numpy as np
import os

# Custom modules
from src.config import VSTConfig
from src.transform import VarianceStabilizer
from src.data_loader import SyntheticGenerator, ImageLoader
from src.metrics import QualityMetrics
from src.codec import BPGCodec

class BPGPipelineApp:
    def __init__(self, bpg_path='bpg-0.9.8-win64', default_path='data/NOISED.tiff'):
        
        # --- Config & State ---
        self.bpg_path = bpg_path
        if not os.path.exists(self.bpg_path):
            print(f"⚠️ WARNING: BPG folder not found at '{self.bpg_path}'. Codec will fail.")
            
        self.codec = BPGCodec(bpg_path)
        
        self.cached_gen_data = None
        self.last_gen_noise = -1
        self.default_original_path = 'data/ORIGINAL.tiff' # Guess path
        
        # --- UI Initialization ---
        self._init_widgets(default_path)
        self._init_layout()
        
    def _init_widgets(self, default_path):
        s = {'description_width': 'initial'}
        layout_half = widgets.Layout(width='48%')
        
        # Source Control
        self.w_source = widgets.Dropdown(options=[('Generator', 'gen'), ('File', 'file')], value='gen', description='Source:', style=s)
        self.w_path = widgets.Text(value=default_path, placeholder='path/to/image.tiff', layout=widgets.Layout(display='none'))
        self.w_noise_gen = widgets.FloatSlider(value=0.25, min=0.01, max=1.0, step=0.01, description='Speckle Lvl:', style=s)
        
        # VST Control
        self.w_a = widgets.FloatSlider(value=8.39, min=1.0, max=20.0, step=0.1, description='VST a:', style=s)
        self.w_b = widgets.FloatSlider(value=1.2, min=1.05, max=5.0, step=0.05, description='VST b:', style=s)
        
        # BPG Control
        # q=0 is lossless (usually), q=51 is worst
        self.w_q = widgets.IntSlider(value=25, min=1, max=51, step=1, description='BPG Quantizer (q):', style=s, continuous_update=False)
        
        self.out_plot = widgets.Output()
        
    def _init_layout(self):
        # Top Row: Data Source
        r1 = widgets.HBox([self.w_source, self.w_noise_gen, self.w_path])
        
        # Middle Row: Parameters
        r2 = widgets.HBox([
            widgets.VBox([widgets.HTML("<b>VST Parameters</b>"), self.w_a, self.w_b], layout=widgets.Layout(border='1px solid #ccc', padding='5px', margin='5px')),
            widgets.VBox([widgets.HTML("<b>Codec Parameters</b>"), self.w_q], layout=widgets.Layout(border='1px solid #ccc', padding='5px', margin='5px'))
        ])
        
        # Events
        self.w_source.observe(self._on_mode_change, names='value')
        for w in [self.w_source, self.w_path, self.w_a, self.w_b, self.w_noise_gen, self.w_q]:
            w.observe(self.update, names='value')
            
        display(widgets.VBox([r1, r2, self.out_plot]))
        self.update()

    def _on_mode_change(self, change):
        if change['new'] == 'gen':
            self.w_path.layout.display = 'none'
            self.w_noise_gen.layout.display = 'flex'
        else:
            self.w_path.layout.display = 'flex'
            self.w_noise_gen.layout.display = 'none'

    def get_data(self):
        if self.w_source.value == 'gen':
            n_lvl = self.w_noise_gen.value
            if self.cached_gen_data is None or n_lvl != self.last_gen_noise:
                self.cached_gen_data = SyntheticGenerator.get_data(n_lvl)
                self.last_gen_noise = n_lvl
            return self.cached_gen_data # (clean, noised)
        else:
            # Load file logic
            try:
                img_in = ImageLoader.load_file(self.w_path.value)
                # Try to find original
                img_clean = None
                if os.path.exists(self.default_original_path):
                    img_clean = ImageLoader.load_file(self.default_original_path)
                return img_clean, img_in
            except:
                return None, None

    def update(self, change=None):
        img_clean, img_noised = self.get_data()
        
        with self.out_plot:
            clear_output(wait=True)
            if img_noised is None:
                print("No image data.")
                return

            # 1. Pipeline: Forward VST
            cfg = VSTConfig(a=self.w_a.value, b=self.w_b.value)
            vst = VarianceStabilizer(cfg)
            
            img_log = vst.forward(img_noised)
            
            # 2. Pipeline: BPG Compression
            try:
                # We compress the Log-Domain image!
                img_log_decoded, f_size, bpp = self.codec.compress_decompress(img_log, q=self.w_q.value)
            except Exception as e:
                print(f"Codec Error: {e}")
                print(f"Check if '{self.bpg_path}' exists and contains bpgenc.exe")
                return
            
            # 3. Pipeline: Inverse VST
            img_final = vst.inverse(img_log_decoded)
            
            # 4. Metrics Calculation
            # Calculate PSNR/SSIM relative to the CLEAN image (if available)
            # or relative to the NOISED image (Input) if no clean ref exists (Reconstruction fidelity)
            
            metrics_text = ""
            metrics_ref = img_clean if img_clean is not None else img_noised
            ref_name = "Ground Truth" if img_clean is not None else "Noisy Input"
            
            val_psnr = QualityMetrics.compute_psnr(metrics_ref, img_final)
            val_ssim = QualityMetrics.compute_ssim(metrics_ref, img_final)
            
            # Calculate metrics in Log Domain (VST space) to see pure compression artifacts
            log_psnr = QualityMetrics.compute_psnr(img_log, img_log_decoded)
            
            # --- Visualization ---
            fig = plt.figure(figsize=(16, 6), constrained_layout=True)
            gs = fig.add_gridspec(2, 4)
            
            # A. Input (Noisy)
            ax_in = fig.add_subplot(gs[0, 0])
            vmin, vmax = np.percentile(img_noised, 1), np.percentile(img_noised, 99)
            ax_in.imshow(img_noised, cmap='gray', vmin=vmin, vmax=vmax)
            ax_in.set_title("1. Noisy Input\n(Multiplicative Noise)")
            
            # B. VST Domain (Before Codec)
            ax_vst = fig.add_subplot(gs[0, 1])
            ax_vst.imshow(img_log, cmap='viridis')
            ax_vst.set_title("2. VST (Log Domain)\nInput to Codec")
            
            # C. VST Domain (After Codec) - Show Artifacts
            ax_dec = fig.add_subplot(gs[1, 1])
            ax_dec.imshow(img_log_decoded, cmap='viridis')
            diff_log = np.abs(img_log - img_log_decoded)
            log_err_mse = np.mean(diff_log**2)
            ax_dec.set_title(f"3. Decoded (Log)\nBPG q={self.w_q.value}, BPP={bpp:.3f}\nLog-MSE: {log_err_mse:.4f}")
            
            # D. Output (Final)
            ax_out = fig.add_subplot(gs[0:2, 2])
            ax_out.imshow(img_final, cmap='gray', vmin=vmin, vmax=vmax)
            ax_out.set_title(f"4. Final Result (Inverse VST)\nComparing to {ref_name}")
            
            # E. Metrics Panel (Text)
            ax_txt = fig.add_subplot(gs[0:2, 3])
            ax_txt.axis('off')
            info = [
                f"Codec: BPG (HEVC)",
                f"Quantizer (q): {self.w_q.value}",
                f"File Size: {f_size / 1024:.2f} KB",
                f"Bit Rate: {bpp:.3f} bits/pixel",
                "-"*20,
                f"Reference: {ref_name}",
                f"PSNR: {val_psnr:.2f} dB",
                f"SSIM: {val_ssim:.4f}",
                "-"*20,
                f"Compression Quality (Log Domain):",
                f"Log-PSNR: {log_psnr:.2f} dB"
            ]
            
            y_pos = 0.9
            for line in info:
                ax_txt.text(0.1, y_pos, line, fontsize=12, fontfamily='monospace')
                y_pos -= 0.08
                
            plt.show()

# Run
app_bpg = BPGPipelineApp(bpg_path='bpg-0.9.8-win64')

In [14]:
import ipywidgets as widgets
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import display, clear_output

# Import custom modules
from src.config import VSTConfig
from src.data_loader import SyntheticGenerator, ImageLoader
from src.codec import BPGCodec
from src.experiments import RateDistortionRunner

class RDAnalysisApp:
    def __init__(self, bpg_path='bpg-0.9.8-win64', default_path='data/NOISED.tiff'):
        self.bpg_path = bpg_path
        self.codec = BPGCodec(bpg_path)
        self.runner = RateDistortionRunner(self.codec)
        
        self.cached_gen = None
        self.last_noise = -1
        self.default_original = 'data/ORIGINAL.tiff'
        
        self._init_ui(default_path)
        
    def _init_ui(self, default_path):
        s = {'description_width': 'initial'}
        
        # Data Config
        self.w_source = widgets.Dropdown(options=[('Generator', 'gen'), ('File', 'file')], value='gen', description='Source:', style=s)
        self.w_path = widgets.Text(value=default_path, placeholder='path/to/image.tiff')
        self.w_noise = widgets.FloatSlider(value=0.25, min=0.01, max=1.0, step=0.01, description='Speckle Lvl:', style=s)
        
        # VST Config
        self.w_a = widgets.FloatSlider(value=8.39, min=1.0, max=20.0, description='a:', style=s)
        self.w_b = widgets.FloatSlider(value=1.2, min=1.05, max=5.0, description='b:', style=s)
        
        # Experiment Range
        self.w_q_start = widgets.IntText(value=10, description='Q Start:', style=s, layout=widgets.Layout(width='150px'))
        self.w_q_end = widgets.IntText(value=51, description='Q End:', style=s, layout=widgets.Layout(width='150px'))
        self.w_q_step = widgets.IntText(value=2, description='Step:', style=s, layout=widgets.Layout(width='150px'))
        
        # Actions
        self.btn_run = widgets.Button(description='▶ Run Analysis', button_style='success', icon='chart-line')
        self.btn_run.on_click(self.run_experiment)
        
        self.prog_bar = widgets.IntProgress(value=0, min=0, max=100, description='Progress:', bar_style='info')
        self.prog_bar.layout.visibility = 'hidden'
        
        self.out_plot = widgets.Output()
        
        # Layout
        ctrl_box = widgets.HBox([self.w_source, self.w_noise, self.w_path])
        vst_box = widgets.HBox([self.w_a, self.w_b])
        range_box = widgets.HBox([self.w_q_start, self.w_q_end, self.w_q_step])
        run_box = widgets.HBox([self.btn_run, self.prog_bar])
        
        # Visibility Logic
        def on_src(change):
            if change['new'] == 'gen':
                self.w_path.layout.display = 'none'
                self.w_noise.layout.display = 'flex'
            else:
                self.w_path.layout.display = 'flex'
                self.w_noise.layout.display = 'none'
        self.w_source.observe(on_src, names='value')
        on_src({'new': 'gen'})
        
        display(widgets.VBox([
            widgets.HTML("<h3>Rate-Distortion Analysis (Q-Curve)</h3>"),
            ctrl_box, 
            widgets.Label("VST Parameters:"), vst_box,
            widgets.Label("Q Range (BPG):"), range_box,
            run_box,
            self.out_plot
        ]))

    def get_data(self):
        # (Same data loading logic as before)
        if self.w_source.value == 'gen':
            val = self.w_noise.value
            if self.cached_gen is None or val != self.last_noise:
                self.cached_gen = SyntheticGenerator.get_data(val)
                self.last_noise = val
            return self.cached_gen
        else:
            try:
                img_in = ImageLoader.load_file(self.w_path.value)
                img_ref = None
                if os.path.exists(self.default_original):
                    img_ref = ImageLoader.load_file(self.default_original)
                return img_ref, img_in
            except: return None, None

    def run_experiment(self, b):
        self.btn_run.disabled = True
        self.prog_bar.layout.visibility = 'visible'
        self.out_plot.clear_output()
        
        try:
            img_ref, img_noised = self.get_data()
            if img_noised is None: 
                print("Error loading data")
                return

            # Defines range of Q
            start, end, step = self.w_q_start.value, self.w_q_end.value, self.w_q_step.value
            q_values = list(range(start, end + 1, step))
            
            # Define Callback for progress bar
            def cb(curr, total):
                self.prog_bar.max = total
                self.prog_bar.value = curr
            
            # RUN PIPELINE
            vst_cfg = VSTConfig(a=self.w_a.value, b=self.w_b.value)
            res = self.runner.run_curve(img_ref, img_noised, vst_cfg, q_values, progress_callback=cb)
            
            # PLOTTING
            with self.out_plot:
                fig, axes = plt.subplots(1, 3, figsize=(18, 5))
                
                # Plot 1: Metrics vs Q (User Request)
                ax1 = axes[0]
                line1, = ax1.plot(res['q'], res['psnr'], 'b-o', label='PSNR')
                ax1.set_xlabel('Quantizer (q)')
                ax1.set_ylabel('PSNR (dB)', color='b')
                ax1.tick_params(axis='y', labelcolor='b')
                ax1.grid(True, alpha=0.3)
                
                # Twin axis for SSIM
                ax1_twin = ax1.twinx()
                line2, = ax1_twin.plot(res['q'], res['ssim'], 'r-s', label='SSIM')
                ax1_twin.set_ylabel('SSIM', color='r')
                ax1_twin.tick_params(axis='y', labelcolor='r')
                
                ax1.set_title("Quality vs. Quantizer (q)")
                # Combine legends
                lines = [line1, line2]
                ax1.legend(lines, [l.get_label() for l in lines], loc='upper right')

                # Plot 2: Rate-Distortion (PSNR vs Bitrate) - The Scientific Standard
                ax2 = axes[1]
                ax2.plot(res['bpp'], res['psnr'], 'g-o')
                ax2.set_xlabel('Bit Rate (bits per pixel)')
                ax2.set_ylabel('PSNR (dB)')
                ax2.set_title("Rate-Distortion Curve\n(Higher & Left is Better)")
                ax2.grid(True, which='both', alpha=0.3)
                # Reverse X axis if you want standard "Rate" look, 
                # but usually BPP increases to the right. 
                
                # Plot 3: File Size vs Q
                ax3 = axes[2]
                ax3.bar(res['q'], res['file_size_kb'], width=step*0.8, color='gray')
                ax3.set_xlabel('Quantizer (q)')
                ax3.set_ylabel('File Size (KB)')
                ax3.set_title("File Size vs. Quantizer")
                ax3.grid(True, axis='y', alpha=0.3)
                
                plt.tight_layout()
                plt.show()
                
        except Exception as e:
            with self.out_plot:
                print(f"Error during experiment: {e}")
        finally:
            self.btn_run.disabled = False
            self.prog_bar.layout.visibility = 'hidden'

# Run App
app_rd = RDAnalysisApp(bpg_path='bpg-0.9.8-win64')